## Section 3: Impacts

In [1]:
#Necessary packages
from ipyleaflet import Choropleth, Map, basemap_to_tiles, LayersControl
import ipywidgets as widgets
from IPython.display import display
from localtileserver import get_leaflet_tile_layer, TileClient
import matplotlib
# from Magics import macro as magics
# from Magics.macro import *
import numpy as np
import pandas as pd
import rasterio #rasterio was preferred over xarray since it is more compatible with ipyleaflet
from rasterio.enums import Resampling
import requests
# import rioxarray as rxr
# import xarray as xr

In [2]:
#custom functions
def resample_raster(path, fact = 0.5, rio = True):
    """
    Resample a raster from its file path
    
    path: str
        Path to raster file
    fact: float
        Upscaling or downscaling factor. Default: 0.5 (i.e. halving the raster spatial resolution)
    """
    # https://rasterio.readthedocs.io/en/stable/topics/resampling.html
    if rio:
        with rasterio.open(path) as dataset:
            # resample data to target shape
            data = dataset.read(
                out_shape=(
                    dataset.count,
                    int(dataset.height * fact),
                    int(dataset.width * fact)
                ),
                resampling = Resampling.bilinear
            )
            # scale image transform
            transform = dataset.transform * dataset.transform.scale(
                (dataset.width / data.shape[-1]),
                (dataset.height / data.shape[-2])
            )
        return(dataset)
    else:
        # method with xarray if needed
        # https://docs.xarray.dev/en/stable/generated/xarray.DataArray.coarsen.html
        pass

In [ ]:
#Parameters to be imported from the main widget
initial_lat_lon = (38, -90)
final_lat_lon = (60, 90)

In [ ]:
#Variable initialization
min_lat = initial_lat_lon[0]
min_lon = initial_lat_lon[1]

max_lat = final_lat_lon[0]
max_lon = final_lat_lon[1]

In [3]:
# Widgets
# widgets selection lat lon
longitude = np.arange(-180,185,5)
longitude_slider = widgets.SelectionRangeSlider(
    options = longitude, index = (0,len(longitude)-1), description='Longitude',
    orientation='horizontal', disabled=False)
latitude = np.arange(-90,95,5)
latitude_slider = widgets.SelectionRangeSlider(
    options = latitude, index = (0,len(latitude)-1), description='Latitude',
    orientation='horizontal', disabled=False)

# widget select sea surface variable
widget_select_ssvar = widgets.Dropdown(description='Sea surface variable', options=('msl', '2t'))

# widget select atm variable
widget_select_atmvar = widgets.Dropdown(description='Atmospheric variable', options=('tp',))
                        # la coma del final és important; sinó agafa cada lletra per separat!!
# widget select timestep: not necessary in this section
# vec_timesteps = list(range(0, len(dates_to_download)))
# widget_sel_timestep = widgets.SelectionSlider(options=vec_timesteps, description='Timestep')

## Population density layer

Raster downloaded from: https://www.worldpop.org/sdi/introapi/ through the code provided in [insert link to code notebook]\
They are too heavy to be downloaded on the go by the dashboard. So, they will be resampled to a lower spatial resolution and only the population in a 500 km radius from the coastline will be kept in the final raster. Right now the spatial resolution is in the order of meters, not needed for our scope. It will be resampled to at least 10km x 10km. Each country will be saved separately. The load function will then gather the last cyclone position and only load the countries in a 500 km radius from the last cyclone position.

## Hazard layers

### Coastal hazard

The return period `rp` could be provided by the user through a widget or we could just select a specific return period and use just that one

In [4]:
def dwnl_coastalhaz(rp):
    """
    Downloads coastal hazard raster maps from The World Bank Data Catalog service
    by providing the hazard return period
    
    rp: str
        Return period.
        Can be one of the following: 5yr, 10yr, 50yr, 100yr, 250yr, 500yr, 1000yr.

    Data source: https://datacatalog.worldbank.org/search/dataset/0038579/Global-coastal-flood-hazard
    """
    links = {
            "5yr": "https://www.geonode-gfdrrlab.org/uploaded/layers/2019/10/16/ss_muis_rp0005m.tif",
            "10yr": "https://www.geonode-gfdrrlab.org/uploaded/layers/2019/10/16/ss_muis_rp0010m.tif",
            "50yr": "https://www.geonode-gfdrrlab.org/uploaded/layers/2019/10/16/ss_muis_rp0050m.tif",
            "100yr": "https://www.geonode-gfdrrlab.org/uploaded/layers/2019/10/16/ss_muis_rp0100m.tif",
            "250yr": "https://www.geonode-gfdrrlab.org/uploaded/layers/2019/10/16/ss_muis_rp0250m.tif",
            "500yr": "https://www.geonode-gfdrrlab.org/uploaded/layers/2019/10/16/ss_muis_rp0500m.tif",
            "1000yr": "https://www.geonode-gfdrrlab.org/uploaded/layers/2019/10/16/ss_muis_rp1000m.tif",
            }
    tif = requests.get(links[rp])
    with open(f"data/impacts/coastalhaz_{rp}.tif", "wb") as tiffile:
            tiffile.write(tif.content)
            #need to compress the tif file when downloading it
            print("Download complete")

def load_coastalhaz(rp, resample = True):
    """
    Loads coastal hazard raster maps downloaded through dwnl_coastalhaz
    by providing the return period. Returns an xarray
    
    rp: str
        Return period.
        Can be one of the following: 5yr, 10yr, 50yr, 100yr, 250yr, 500yr, 1000yr.
    resample: bool
        If True, the raster is resampled at a lower spatial resolution
    """
    path = f"data/impacts/coastalhaz_{rp}.tif"
    if resample:
        coh = resample_raster(path)
    else:
        coh = rasterio.open(path)
    return(coh)

def plot_coastalhaz(coh, cut = False):
    """
    Plot coastal hazard raster provided as a layer in an ipyleaflet map
    """
    client = TileClient(coh)
    t = get_leaflet_tile_layer(client, opacity = 0.65) #, palette = "Fall_7") #fix the plotting palette
    m = Map(center=client.center(), zoom=client.default_zoom)
    m.add_layer(t)
    m.add_control(LayersControl())
    m.layout.height="700px"
    return(m)

In [21]:
#Download the 50yr coastal hazard raster map
# dwnl_coastalhaz("50yr")
#Load the 50yr coastal hazard raster map as an xarray
coh = load_coastalhaz("50yr")
print(coh)

<xarray.DataArray (band: 1, y: 16744, x: 43200)>
[723340800 values with dtype=float32]
Coordinates:
  * band         (band) int32 1
  * x            (x) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * y            (y) float64 83.62 83.61 83.6 83.6 ... -55.88 -55.89 -55.9 -55.9
    spatial_ref  int32 0
Attributes:
    AREA_OR_POINT:  Area
    DataType:       Generic
    _FillValue:     0.0
    scale_factor:   1.0
    add_offset:     0.0


In [ ]:
plot_coastalhaz(coh)

### Cyclone hazard

The return period `rp` could be provided by the user through a widget or we could just select a specific return period and use just that one

In [6]:
def dwnl_cyclonehaz(rp):
    """
    Downloads cyclone hazard raster maps from The World Bank Data Catalog service
    by providing the hazard return period

    rp: str
        Return period.
        Can be one of the following: 50yr, 100yr, 250yr, 500yr, 1000yr.
    
    Data source: https://datacatalog.worldbank.org/search/dataset/0038577/Global-cyclone-hazard
    """
    links = {
            "50yr": "https://www.geonode-gfdrrlab.org/uploaded/layers/2019/02/26/viento_mundo_tr50_int1_Ai5zJup.tif",
            "100yr": "https://www.geonode-gfdrrlab.org/uploaded/layers/2019/02/26/viento_mundo_tr100_int1_afg5XsL.tif",
            "250yr": "https://www.geonode-gfdrrlab.org/uploaded/layers/2019/02/26/viento_mundo_tr250_int1.tif",
            "500yr": "https://www.geonode-gfdrrlab.org/uploaded/layers/2019/02/26/viento_mundo_tr500_int1.tif",
            "1000yr": "https://www.geonode-gfdrrlab.org/uploaded/layers/2019/02/26/viento_mundo_tr1000_int1.tif",
            }
    tif = requests.get(links[rp])
    with open(f"data/impacts/cyclonehaz_{rp}.tif", "wb") as tiffile:
            tiffile.write(tif.content)
            print("Download complete")

def load_cyclonehaz(rp):
    """
    Loads coastal hazard raster maps downloaded through dwnl_coastalhaz
    by providing the return period. Returns an xarray

    rp: str
        Return period.
        Can be one of the following: 50yr, 100yr, 250yr, 500yr, 1000yr.
    """
    path = f"data/impacts/cyclonehaz_{rp}.tif"
    cyh = rasterio.open(path)
    return(cyh)

def plot_cyclonehaz(cyh):
    """
    Plots the cyclone hazard layer provided as input in an ipyleaflet map
    """
    client = TileClient("data/impacts/cyclonehaz_50yr.tif")
    t = get_leaflet_tile_layer(client, opacity = 0.65) #, palette = "Fall_7") #fix the plotting palette
    m = Map(center=client.center(), zoom=client.default_zoom)
    m.add_layer(t)
    m.add_control(LayersControl())
    m.layout.height="700px"
    return(m)

In [7]:
#Download the 50yr cyclone hazard raster map
# dwnl_cyclonehaz("50yr")
#Load the 50yr cyclone hazard raster map as an xarray
cyh = load_cyclonehaz("50yr")

In [8]:
rasterio.crs.CRS({"init": "EPSG:4326"})

CRSError: The EPSG code is unknown. PROJ: proj_create_from_database: C:\Users\paolo\anaconda3\Library\share\proj\proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.

In [4]:
cyh.crs

NameError: name 'cyh' is not defined

In [9]:
plot_cyclonehaz(cyh)

CRSError: The EPSG code is unknown. PROJ: proj_create_from_database: C:\Users\paolo\anaconda3\Library\share\proj\proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.

### World Risk Index
From the data of the World Risk Index, retrieve the expositions to coastal floodings, tsunamis and sea level rise of each country.\

In [26]:
def dwnl_riskidx():
    """
    Downloads the World Risk Index for 2022

    Data source: https://data.humdata.org/dataset/1efb6ee7-051a-440f-a2cf-e652fecccf73
    """
    links = {
     #     "meta": "https://data.humdata.org/dataset/1efb6ee7-051a-440f-a2cf-e652fecccf73/resource/9ab4cdc6-9682-45a6-8314-db5e0d49e7a7/download/worldriskindex-meta.xlsx",
         "data": "https://data.humdata.org/dataset/1efb6ee7-051a-440f-a2cf-e652fecccf73/resource/1b47b40c-f746-427c-bbf8-8caa157e03da/download/worldriskindex-2022.csv",
     #     "trend": "https://data.humdata.org/dataset/1efb6ee7-051a-440f-a2cf-e652fecccf73/resource/9ae68502-6011-4276-a99d-118bb2826323/download/worldriskindex-trend.csv"
    }
    for id in links:
        f = requests.get(links[id])
        if id == "meta":
             fformat = "xlsx"
        else:
             fformat = "csv"
        with open(f"data/impacts/worldriskindex22_{id}.{fformat}", "wb") as file:
                file.write(f.content)
        print("World Risk Index ", id, " downloaded")

def load_riskidx():
     """
     Load data in a DataFrame and keep only columns with the annual percentage of people of that
     country exposed to severe tsunamis, severe coastal floods and sea level rise.
     """
     csv = pd.read_csv("data/impacts/worldriskindex22_data.csv")
    #  cols = ["Country", "ISO3", "EI_02d_Base", "EI_03d_Base", "EI_07b_Base"]
     cols = ["Country", "ISO3", "EI_02d_Norm", "EI_03d_Norm", "EI_07b_Norm"]
     csv = csv.loc[:, cols]
     cols = ["Country", "ISO3", "Tsunamis", "Coastal_floods", "Sea_level_rise"]
     csv.columns = cols
     return(csv)

def plot_riskidx(var, csv = None):
    # https://ipyleaflet.readthedocs.io/en/latest/layers/choropleth.html
    # https://coderzcolumn.com/tutorials/data-science/choropleth-maps-using-ipyleaflet
    #Load risk data
    if csv is None: csv = load_riskidx()
    #Load world countries boundaries
    #from: https://public.opendatasoft.com/explore/dataset/world-administrative-boundaries/export/
    f = requests.get("https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/world-administrative-boundaries/exports/geojson?lang=en&timezone=Europe%2FBerlin")
    geo_json_data = json.loads(f.content)
    for d in geo_json_data["features"]:
          d["iso"] = d["properties"]["iso3"]
    mapping  = dict(zip(csv["ISO3"].str.strip(), csv[var]))
    for d in geo_json_data["features"]:
        if d["iso"] not in mapping:
            mapping[d["iso"]] = 0 #check if nan is possible
    m = Map(zoom=2)
    layer = Choropleth(
                    geo_data = geo_json_data,
                    choro_data = mapping,
                    # colormap="blues",
                    style = {'fillOpacity': 1.0, "color":"black"},
                    key_on = "iso")
    m.add_layer(layer)
    m.add_control(LayersControl())
    m.layout.height="700px"
    return(m)

**Plots**:\
They show the percentage of the population exposed to the hazard considered normalized across the countries

In [31]:
m = plot_riskidx("Tsunamis")
m

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [6]:
m = plot_riskidx("Coastal_floods")
m

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [7]:
m = plot_riskidx("Sea_level_rise")
m

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

### Combined map

Plot all the layers above in a unique ipyleaflet